In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
from pprint import pprint
from typing import Literal

repo_root = str(Path.cwd().parent.parent)
if repo_root not in sys.path:
    sys.path.append(repo_root)

import cv2
from PIL import Image
from sam2util import convert_images_to_mp4
from tqdm import tqdm

from model.common import crop_driver_image_contains
from model.depth import convert_video_to_depth


In [4]:
ROOT = Path().home() / 'source/driver-dataset/2024-10-28-driver-all-frames'
assert ROOT.exists()

CATEGORIES = ['normal', 'anomal']
RESIZE = (518, 518)

In [4]:
def process_image(image_path: Path, target_dir: Path) -> None:
    image = Image.open(image_path)
    image = crop_driver_image_contains(image, image_path)
    image = image.resize(RESIZE)
    target_path = target_dir / image_path.name
    image.save(target_path)


def crop_frames(
    root_dir: Path,
    source: str = 'images',
    target: str = 'crop_rgb',
    image_extension: str = 'jpg',
    max_workers: int | None = None,
) -> None:
    source_directories = sorted(
        [d / cat / source for d in root_dir.iterdir() for cat in CATEGORIES]
    )
    assert all(d.exists() for d in source_directories)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for source_dir in (pbar := tqdm(source_directories)):
            pbar.set_postfix_str(str(source_dir))
            image_paths = sorted(source_dir.glob(f'*.{image_extension}'))
            target_dir = source_dir.parent / target
            target_dir.mkdir(exist_ok=True)

            for image_path in image_paths:
                futures.append(executor.submit(process_image, image_path, target_dir))

        for future in tqdm(futures, desc='Processing images'):
            future.result()  # Ensure exceptions are raised if any


In [ ]:
# ~14 minutes
crop_frames(ROOT)

In [5]:
def convert_frames_to_video(
    base_directory: str | Path,
    source_type: str,
    preset: Literal[
        'ultrafast',
        'superfast',
        'veryfast',
        'faster',
        'fast',
        'medium',
        'slow',
        'slower',
        'veryslow',
    ] = 'medium',
    crf: int = 0,
    image_extension: str = 'jpg',
) -> None:
    """Convert extracted frames to video. Assumes that function `extract_frames` has been called."""
    base_dir = Path(base_directory)
    if not base_dir.exists():
        raise ValueError(f'Session directory not found: {base_dir}')

    all_sequences = sorted([p for p in base_dir.rglob(source_type) if p.is_dir()])
    pprint(all_sequences)

    for seq_dir in (pbar := tqdm(all_sequences)):
        pbar.set_postfix_str(seq_dir.parent.name)
        output_file = seq_dir / 'video.mp4'
        convert_images_to_mp4(
            image_folder=seq_dir,
            output_video_path=output_file,
            preset=preset,
            crf=crf,
            image_format=image_extension,
        )


In [ ]:
# ~14 minutes
convert_frames_to_video(ROOT, source_type='crop_rgb', preset='slow', crf=10)

In [8]:
# 2500 frames --> 56 GB of RAM
def _split_video(
    input_path: str | Path, output_dir: str | Path, max_frames: int = 1000
) -> None:
    # Open the video file
    cap = cv2.VideoCapture(str(input_path))

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    num_parts = total_frames // max_frames + 1
    max_frames = total_frames // num_parts + 1
    print(
        f"Splitting video '{input_path}' into {num_parts} parts with {max_frames} frames each."
    )

    # Ensure output directory exists
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)

    part_num = 1
    frame_count = 0
    out = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # End of video

        # If starting a new chunk, create a new video writer
        if frame_count % max_frames == 0:
            if out:  # Release previous writer
                out.release()
            output_path = output_dir / f'part_{part_num:03d}.mp4'
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
            out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))
            part_num += 1

        # Write frame to current output video
        out: cv2.VideoWriter
        out.write(frame)
        frame_count += 1

    # Release resources
    cap.release()
    if out:
        out.release()

    print(f"Splitting completed. {part_num - 1} parts saved in '{output_dir}'.")


def split_videos(base_dir: str | Path, source_type: str) -> None:
    base_dir = Path(base_dir)
    if not base_dir.exists():
        raise ValueError(f'Session directory not found: {base_dir}')

    all_sequences = sorted([p for p in base_dir.rglob(source_type) if p.is_dir()])
    all_videos = sorted(
        [p / 'video.mp4' for p in all_sequences if (p / 'video.mp4').exists()]
    )
    pprint(all_videos)

    for video_path in (pbar := tqdm(all_videos)):
        pbar.set_postfix_str(str(video_path))
        output_dir = video_path.parent / 'video_parts'
        output_dir.mkdir(exist_ok=True)
        _split_video(video_path, output_dir)

In [ ]:
# Example usage
split_videos(ROOT, 'crop_rgb')

In [ ]:
# ~160 minutes
convert_video_to_depth(ROOT, source_type='video_parts', use_parent_dir=True)